In [5]:
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange
from heapdict import heapdict
from razdel import sentenize

C:\Users\Admin\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
base_url = 'https://burunen.ru/bur'
downloaded_pages = dict()
urls_queue = {base_url}

for i in trange(10_000):
    if not urls_queue:
        break
    url = urls_queue.pop()
    if url.startswith('https://burunen.ru/bur'):
        html = requests.get(url).text
        downloaded_pages[url] = html
        soup = BeautifulSoup(html)
        for a in soup.findAll('a'):
            if not a.has_attr('href'):
                # skip anchors
                continue
            href = a['href']
            if href.startswith('/bur/'):
                new_url = 'https://burunen.ru' + href
                if new_url not in urls_queue:
                    urls_queue.add(new_url)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [41:05<00:00,  4.06it/s]


In [7]:
len(downloaded_pages)

697

In [8]:
parsed_pages = []

for url in tqdm(downloaded_pages):
    if '?start=' in url:
        continue
    html = downloaded_pages[url]
    soup = BeautifulSoup(html)
    try:
        # title = soup.find('div', {'class': 'header'}).text.strip()
        # category = soup.find('dd', {'class': 'b-tags t-caption'}).find('a').text
        body = soup.find('div', {'itemprop': 'articleBody'}).text
        body = body.replace('\n', '')
        body = body.replace('©', '')
    except AttributeError as e:
        # print(url)
        continue
    parsed_pages.append({
        'url': url,
        # 'title': title, 
        # 'category': category,
        'body': body
    })

100%|████████████████████████████████████████████████████████████████████████████████| 697/697 [00:10<00:00, 67.38it/s]


не подходят "главные" страницы, страницы с текстами парсятся норм

In [9]:
print(len(parsed_pages))

623


In [10]:
df = pd.DataFrame(parsed_pages)

In [14]:
df

,url,body
0,https://burunen.ru/bur/news/society/97050-zhar...,Уг гарбал Тогочи Данзанов 1903 ондо Ман­далай ...
1,https://burunen.ru/bur/news/society/92184-molo...,"Бултанда мэдээжэ ко­видай шалтагһаа, хоёр жэлэ..."
2,https://burunen.ru/bur/news/agriculture/96783-...,"фото: ""Гордость Бурятии"" гэһэн портал Буряад ..."
3,https://burunen.ru/bur/news/sports/97330-sagaa...,фото: Борис Балданов Мүрысөөндэ Буряадай бүхы...
4,https://burunen.ru/bur/news/culture/97467-baya...,Хотын 33-дахи гимназиин багша Баярма Бадмаева ...
...,...,...
618,https://burunen.ru/bur/news/society/93666-erkh...,фото: Буряад Уласай засагай сайтһаа Харин сен...
619,https://burunen.ru/bur/news/politics/93358-zos...,фото: Буряад Уласай засагай сайтһаа Энэ хабар...
620,https://burunen.ru/bur/news/society/93522-bayg...,фото: tunkapark.ru сайтһаа зураг. Эрдэмэй бол...
621,https://burunen.ru/bur/news/culture/93261--/,Бүхыдөө зүжэг монологуудһаа бүридэнэ. Иимэ түх...


In [12]:
bur_text = df['body']

In [17]:
bur_text = bur_text.tolist()

In [21]:
import re

In [22]:
def split_sentences(text):
    split_text = [] 
    string = ''
    for i in text:
        string += i + ' '
    reg = r'(?<=[\.\?\!])'
    # reg = r'(?<=[\.\?\!])(?!(»|“))'
    new_lines = re.split(reg, string)
    for line in new_lines:
        if line is not None:
            if re.findall(r'(\w|[«„\d])+', line)!=[] and re.findall(r'(\d\:\d)+', line)==[]:
                split_text.append(line.lstrip())
    return split_text

In [23]:
bur_sentences = split_sentences(bur_text)

In [28]:
bur_sents = []
for sent in bur_sentences:
    bur_sents.append(sent.replace('\xad', ''))

In [30]:
bur_sents[:10]

['Уг гарбал Тогочи Данзанов 1903 ондо Мандалай эбэр гэжэ hууринда Раднын Данзанай гэр бүлэдэ түрэhэн юм.',
 'Уг изагуур гэхэдэ, булагад буряад, готолбуумал угай буянгууд омогтон болоно.',
 'Угай бэшэгтэ иимэ байна: Аhа-Батури-Тармай-Бэшэгтэ-Улзойто-Ганжур-Радна-Данзан-Тогочи.',
 'Бага наhан Суусагайн нуурай эрьеэр наадагша бэлэй.',
 'Бүдэ эгэшэтэеэ үншэржэ, яhалашье зобоо туляа юм ааб даа.',
 'Шадалтай баян айлаар ажал хэдэг hэн.',
 'Тогочи hүбэлгэн шуран, эрмэлзэл ехэтэй хүбүүн байгаа.',
 'Һамбаа олоод лэ, элдэб ажал зохёохо, хэхэ.',
 'Дабирхай нэрэжэ, хоолойгоо тэжээдэг болоо hэн.',
 'Урдандаа ехэ хэрэглэгдэдэг байhан гээшэ: тэргэ дабирхайдаха, малшье эмшэлдэг байгаа.']

In [31]:
with open('buryad_unen.txt', 'w', encoding = 'utf-8') as f:
    for sent in bur_sents:
        f.write("%s\n" % sent)

In [2]:
with open('buryad_unen.txt', 'r', encoding = 'utf-8') as f:
    buryad_unen = f.readlines()

In [3]:
buryad_unen = ' '.join(buryad_unen)

In [6]:
sents = list(sentenize(buryad_unen))
bur_text = []
for substr in sents:
    bur_text.append(substr.text.replace('\n', ''))

In [7]:
len(bur_text)

20891

In [8]:
with open('buryad_unen.txt', 'w', encoding = 'utf-8') as f:
    for sent in bur_text:
        f.write("%s\n" % sent)